# 🎧 Spotify MP3 Converter

Converte músicas, álbuns e playlists do **Spotify** para arquivos **MP3** automaticamente, utilizando o **YouTube** como fonte — tudo direto no **Google Colab**.

---

👤 **Autor:** Kira_Ghoul  
📅 **Projeto:** Spotify MP3 Converter  
💻 **Plataforma:** Google Colab  

---

⚠️ **Aviso Legal:** Este projeto é apenas para fins educacionais.  
Respeite os direitos autorais das plataformas envolvidas.

In [14]:
# ==========================================
# ⚡ Spotify → YouTube MP3 Downloader (Colab)
# ==========================================

!pip install -q yt-dlp spotipy mutagen rapidfuzz

# ---------- Imports ----------
import os, shutil, glob, requests, queue, threading
import yt_dlp, spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from rapidfuzz import fuzz
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, TIT2, TPE1, TALB, APIC
from google.colab import files
from time import sleep

# ---------- Spotify Credentials ----------
SPOTIFY_CLIENT_ID = 'a329b4abe42c46c8928c082816bc8d36'
SPOTIFY_CLIENT_SECRET = '2d23bae3182b44f38662abf837a26fe7'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET
))

# ---------- Cookies opcional ----------
cookies_path = None
print("📁 Opcional: faça upload do arquivo cookies do YouTube (cookies.txt).")
print("   Ou clique em 'Cancelar' para continuar sem cookies.")
try:
    uploaded = files.upload()
    if uploaded:
        cookies_path = list(uploaded.keys())[0]
        print(f"✅ Cookies carregados: {cookies_path}")
except:
    print("⚠️ Nenhum cookies enviado. Continuando sem cookies...")

# ---------- Variável global para contador ----------
counter_lock = threading.Lock()
global_index = 1  # contador global das músicas

# ---------- Funções auxiliares ----------
def tag_mp3(path, title, artist, album, img_url=None):
    """Adiciona tags e capa à MP3"""
    try:
        audio = MP3(path, ID3=ID3)
        audio.add_tags()
    except:
        pass
    audio.tags.add(TIT2(encoding=3, text=title))
    audio.tags.add(TPE1(encoding=3, text=artist))
    audio.tags.add(TALB(encoding=3, text=album))
    if img_url:
        try:
            img = requests.get(img_url, timeout=5).content
            audio.tags.add(APIC(encoding=3, mime='image/jpeg', type=3, desc='Cover', data=img))
        except:
            pass
    audio.save()

def simple_match(track_name, yt_title):
    """Verifica se o título bate razoavelmente"""
    return fuzz.partial_ratio(track_name.lower(), yt_title.lower()) > 70

def download_worker(q, folder):
    """Thread worker para baixar músicas com numeração global"""
    global global_index

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{folder}/%(title)s.%(ext)s',
        'quiet': True,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }]
    }

    # Adiciona cookies só se existirem
    if cookies_path:
        ydl_opts['cookies'] = cookies_path

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        while True:
            try:
                track = q.get(timeout=2)
            except queue.Empty:
                break

            title = track['name']
            artist = track['artists'][0]['name']
            album = track['album']['name']
            cover = track['album']['images'][0]['url'] if track['album']['images'] else None
            q_str = f"{title} {artist}"

            # Número da música de forma thread-safe
            with counter_lock:
                index = global_index
                global_index += 1

            print(f"🎵 Baixando {index}º: {title} - {artist}")

            try:
                results = ydl.extract_info(f"ytsearch3:{q_str}", download=False)
                entry = next((e for e in results['entries'] if simple_match(title, e['title'])), results['entries'][0])
                ydl.download([entry['webpage_url']])
                mp3 = max(glob.glob(f"{folder}/*.mp3"), key=os.path.getmtime)
                tag_mp3(mp3, title, artist, album, cover)
            except yt_dlp.utils.DownloadError as e:
                if "Sign in to confirm you're not a bot" in str(e) and not cookies_path:
                    print("⚠️ YouTube bloqueou. Faça upload do cookies.txt para continuar.")
                else:
                    print(f"   ⚠️ {title}: {e}")
            finally:
                q.task_done()

# ---------- Função principal ----------
def download_spotify(url, max_threads=5):
    """Baixa músicas de Spotify via YouTube no Colab"""
    # Detecta tipo de link
    if "playlist" in url:
        info = sp.playlist(url)
        name = info['name']
        results = sp.playlist_items(url)
        tracks = []
        while results:
            tracks.extend([i['track'] for i in results['items'] if i['track']])
            results = sp.next(results)
    elif "album" in url:
        info = sp.album(url)
        name = info['name']
        tracks = info['tracks']['items']
        for t in tracks:
            t['album'] = info
    elif "track" in url:
        info = sp.track(url)
        name = info['name']
        tracks = [info]
    else:
        print("❌ Link Spotify inválido.")
        return

    print(f"\n🎧 Encontradas {len(tracks)} músicas em '{name}'\n")

    # Cria pasta de saída
    folder = f"/content/{name.replace('/', '_')}"
    os.makedirs(folder, exist_ok=True)

    # Fila de download
    q = queue.Queue()
    for t in tracks:
        q.put(t)

    # Threads
    num_threads = min(max_threads, len(tracks))
    print(f"🚀 Iniciando {num_threads} threads de download...\n")
    threads = []
    for _ in range(num_threads):
        t = threading.Thread(target=download_worker, args=(q, folder))
        t.start()
        threads.append(t)
        sleep(0.5)

    for t in threads:
        t.join()

    # Compacta em ZIP
    zip_path = f"/content/{name.replace('/', '_')}.zip"
    shutil.make_archive(zip_path[:-4], 'zip', folder)
    print(f"\n✅ Downloads completos! ZIP: {zip_path}")

    # Download no Colab
    files.download(zip_path)
    shutil.rmtree(folder, ignore_errors=True)

# ---------- Executar ----------
spotify_link = input("🎵 Cole o link da playlist/álbum/música do Spotify: ").strip()
download_spotify(spotify_link, max_threads=5)

📁 Opcional: faça upload do arquivo cookies do YouTube (cookies.txt).
   Ou clique em 'Cancelar' para continuar sem cookies.


Saving www.youtube.com_cookies.txt to www.youtube.com_cookies (2).txt
✅ Cookies carregados: www.youtube.com_cookies (2).txt
🎵 Cole o link da playlist/álbum/música do Spotify: https://open.spotify.com/playlist/6wdMv8fBts59MRftXF8Npu?si=493qNqOaQPKV4fyXzLaq-A&pi=l6_85titTE2hs

🎧 Encontradas 52 músicas em 'Tuga Hit'

🚀 Iniciando 5 threads de download...

🎵 Baixando 1º: Pensamento - Plutonio
🎵 Baixando 2º: Europa (feat. Deejay Telio, Teto & Gson) - Mizzy Miles
🎵 Baixando 3º: Cara Que Engana - Deejay Telio
🎵 Baixando 4º: I'm Sorry - Mizzy Miles
🎵 Baixando 5º: PRA MIM - T-Rex
🎵 Baixando 6º: Não Sinto - Wet Bed Gang
🎵 Baixando 7º: Marcas De Bikini - LEO2745
🎵 Baixando 8º: Não Tou Nem Aí - Plutonio
🎵 Baixando 9º: 300 - Wet Bed Gang
🎵 Baixando 10º: Sr. Guarda - Plutonio
🎵 Baixando 11º: ¡ +1 ! - Papillon
🎵 Baixando 12º: DISTÂNCIA - T-Rex
🎵 Baixando 13º: Sweet Spot - Papillon
🎵 Baixando 14º: Aviola II - Bispo
🎵 Baixando 15º: Tempo - FRANKIEONTHEGUITAR
🎵 Baixando 16º: 1 de Abril - Plutonio
🎵 Baixa

Exception in thread Thread-4 (download_worker):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2088028037.py", line 110, in download_worker
  File "/tmp/ipython-input-2088028037.py", line 50, in tag_mp3
UnboundLocalError: cannot access local variable 'audio' where it is not associated with a value


🎵 Baixando 36º: Chaminé (Zara G) - Wet Bed Gang
🎵 Baixando 37º: Protagonista - Dillaz
🎵 Baixando 38º: Água de Coco - ProfJam
🎵 Baixando 39º: Não Tens Visto - Wet Bed Gang
🎵 Baixando 40º: Impossíveis - Piruka
🎵 Baixando 41º: Mesa 8 - Wet Bed Gang
🎵 Baixando 42º: Se Não Acordar Amanhã - Piruka
🎵 Baixando 43º: Sons de Amor - ANDREZO
🎵 Baixando 44º: Prova dos 9 - Piruka
🎵 Baixando 45º: Tou Bem (feat. Lhast) - ProfJam
🎵 Baixando 46º: Prada - Plutonio
🎵 Baixando 47º: Grillz (Zizzy Jr) - Wet Bed Gang
🎵 Baixando 48º: Lisabona - Plutonio
🎵 Baixando 49º: Bolero - X-Tense
🎵 Baixando 50º: After Party - Deejay Telio
🎵 Baixando 51º: Cafeína - DJ Dadda
🎵 Baixando 52º: 23 de Maio - Wet Bed Gang

✅ Downloads completos! ZIP: /content/Tuga Hit.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>